In [6]:
class Parent:
    def __init__(self, value):
        self.value = value
        print(f"Parent initialized with value {self.value}")

class Child(Parent):
    def __init__(self, value, extra):
        super().__init__(value)  # 调用父类的 __init__ 方法
        self.extra = extra
        print(f"Child initialized with value {self.value} and extra {self.extra}")


In [7]:
Child.value = 10
Child.extra = 20


c=Child(10,20)

Parent initialized with value 10
Child initialized with value 10 and extra 20


## Tensor 张量

In pytorch, the core data type is tensor, which is a multi-dimensional matrix containing elements of a single data type. The core operation associated with this data type is gradient calculation. In this notebook, we will introduce some basic concepts and code operations related to it.

在PyTorch框架中的一个核心数据类型是张量，即Tensor。伴随此数据类型的核心操作是梯度计算。下面将逐一说明与之相关一些基本概念和代码操作。

### 1. Creating Tensor 张量的创建

In [2]:
import torch
import numpy as np

t0 = torch.zeros(5, 3, 2)                          #各元素用0填充
t1 = torch.ones(5, 3, 2)                           #各元素用1填充
tr = torch.randn(5, 3, 2)                          #各元素用标准正态分布随机数填充
lis2t = torch.tensor([[1,2],[3,4],[5,6]])          # 直接由列表数据构造张量
ta = torch.arange(12.0)                            # 由浮点型序列构造张量
arr2t = torch.tensor(np.array([[1,2],[3,4]]))      # 将np数组转化成张量
mat2t = torch.tensor(np.matrix([[1,3],[3,5]]))     # 将np矩阵转换成张量
t2mat = np.matrix(mat2t)                           # 将张量转换成np矩阵
t2arr = np.array(arr2t)                            # 将张量转换成np数组


### 2. Tensor Attributes 张量的属性

In [3]:
s0 = t0.shape                                      # 张量的形状
device0 = t0.device                                # 张量的运算设备（如GPU或CPU）
print(t0.requires_grad)                            # 当前张量是否会被视为自变量来计算梯度
print(t0.is_leaf)                                  # 当前张量是否为叶子节点（即手创或由“不计算梯度”张量运算出的张量）
print(t0.grad_fn)                                  # 生成当前张量的生成函数（凡叶子节点的生成函数都为None）


False
True
None


### 3. Tensor Operations 张量的操作

In [4]:
ele = tr[0][1][1].item()                           # 取张量中的一个元素。注意：只有标量才能取其值
tta = ta.reshape(2, 2, 3)                          # 更改张量形状，但元素数要与原张量一致
arr2ft = arr2t.float()                             # 将张量中各元素类型转为浮点型，许多张量运算只适用于浮点型
y1 = arr2ft * mat2t                                # 得到两张量的对应元素乘积
y2 = arr2ft @ arr2ft                               # 对2阶张量（矩阵）做点积
y3 = torch.t(arr2t)                                # 对2阶以下张量求其转置
y4 = torch.inverse(arr2ft)                         # 对2阶张量求其逆张量（张量中元素必须是浮点型）
m2 = y2.mean()                                     # 求张量中各元素均值（也只适用于浮点型元素）


### 4. Tensor Gradient 张量的梯度

In [7]:
arr2ft.requires_grad = True                        # 将要计算梯度的张量的“计算梯度”属性置为真，默认是假
yy = arr2ft @ arr2ft                               # 做矩阵乘法
yy.requires_grad                                   # “计算梯度”属性置真后算得的张量，其该属性也自动为真
print(yy.is_leaf)                                  # 由“计算梯度”张量运算出的新张量不再是叶子节点
print(arr2ft.is_leaf)                              # 手创的“计算梯度”张量是叶子节点
print(yy.grad_fn)                                  # 非叶子张量会有生成函数
fy = yy.mean()                                     # 由张量运算出标量，因为只能针对作为标量的最终函数值计算梯度
print(arr2ft.grad)                                 # 梯度计算前叶子张量的梯度属性为空
fy.backward()                                      # 利用偏导的反向传递计算各叶子张量的梯度
print(arr2ft.grad)                                 # 每次算得的梯度会累加进叶子张量的梯度属性里


False
True
None
tensor([[-0.3750,  0.3750],
        [-0.3750,  0.3750]])


In PyTorch framework, no matter how complex the function operation is, whether it is a library function or a handwritten one, as long as a scalar function value can be obtained in the end, the gradient of this function value to each leaf tensor can be traced back. The mathematical principle behind it is the partial derivative calculation formula of multivariate functions.

在PyTorch框架中，各叶子张量无论经历怎样复杂的函数运算，这些函数也无论是库函数还是手写的，只要最终能得到一个标量函数值，都能追溯到此函数值对各叶子张量的梯度，其背后的数学原理就是多元函数的偏导计算公式。

### 5. 更新叶子张量后使其仍保持叶子属性

前面提到叶子节点必须是手创或是全由"不计算梯度"的张量运算出的张量，但在神经网络训练过程中，待训练的参数张量一方面必须要设置成是“计算梯度”的，另一方面在每次训练中要被迭代更新，而这更新过程就是由原参数张量参与计算得到新参数张量，这样一来，新参数张量必然不再是叶子节点，也就无法再进行下一次训练迭代了。为解决此问题，需要设法使更新后的叶子张量仍保持其叶子属性。此时要用到Python的上下文操作：

In [6]:
with torch.no_grad():
    arr2ft = arr2ft - arr2ft.grad                   # 在no_grad上下文中更新叶子张量
print(arr2ft.is_leaf)                               # 此时原叶子张量虽被运算更新，但仍恢复成最初叶子属性
arr2ft.requires_grad                                # 梯度计算开关也置回默认关闭状态
print(arr2ft.grad)                                  # 梯度属性也自动清空
arr2ft = arr2ft - arr2ft.grad                       # 如果直接更新叶子张量，则会将其变为生成张量，即非叶子张量
print(arr2ft.is_leaf)


True
None


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

# Pytorch包

### 1. nn.Module

Every layer is nn.Module. It is a base class for all neural network modules. Your models should also subclass this class.

每一个层都是nn.Module。它是所有神经网络模块的基类。你的模型也应该是这个类的子类。

And nn.Module is nested, which means you can put an nn.Module inside another nn.Module.

并且nn.Module是嵌套的，这意味着你可以把一个nn.Module放在另一个nn.Module里。

#### 1.1 nn.Module提供的方法

In [4]:
import torch.nn as nn
import torch.nn.functional as F


In [ ]:

#nn.Module提供了一些方法，比如：
"""
nn.Linear() #线性变换
nn.Conv2d() #二维卷积
nn.BatchNorm2d() #批标准化
nn.ReLU() #激活函数
nn.Sigmoid() #激活函数
nn.MaxPool2d() #最大池化
nn.CrossEntropyLoss() #交叉熵损失
nn.MSELoss() #均方误差损失
nn.Sequential() #序列容器
nn.ModuleList() #模块列表
nn.ModuleDict() #模块字典
nn.functional #常用函数库
nn.Dropout() #Dropout层
nn.ConvTranspose1d()  #一维转置卷积
"""
#等等


#### 1.2 Container

nn.Sequential的作用是将网络的层组合到一起，以形成一个新的网络。

这样可以简化网络的结构，使得网络的结构更加清晰，不必在forward函数中一层一层的写网络结构。

In [5]:
#对比以下代码，可以看出nn.Module的优势

# 1. 无nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)  # 直接使用functional API调用ReLU
        x = self.fc2(x)
        x = F.relu(x)  # 再次使用functional API调用ReLU
        return x
    

# 2. 有nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        return x

推荐使用第一种方法，即将激活函数作为模块来使用。这种方式使得模型的结构更加清晰，并且更容易进行模块的重用和管理。直接调用激活函数（如F.relu）虽然在编写代码时可能更快，但在处理复杂模型时可能会减少代码的可读性和可维护性。

In [ ]:
# 对比以下代码，可以看出nn.sequential的优势

# 1. 无nn.sequential

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #需要给每个层命名
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 64, 5)
        self.fc1 = nn.Linear(64 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# 2. 有nn.sequential
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #不需要给每个层命名
        self.conv = nn.Sequential(
            nn.Conv2d(1, 20, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(20, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 5 * 5, 500),
            nn.ReLU(),
            nn.Linear(500, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 64 * 5 * 5)
        x = self.fc(x)
        return x

#### 1.3 Parameters

In [ ]:
# 通过net.parameters()方法可以查看模型的参数
net = Net()
for param in net.parameters():
    print(param)
# 通过net.named_parameters()方法可以查看模型的参数及对应的名字
for name, param in net.named_parameters():
    print(name, param.size())

# 通过list()方法可以查看模型的参数
print(list(net.parameters()))
# 通过dict()方法可以查看模型的参数及对应的名字
print(dict(net.named_parameters()))


#### 1.4 模块的节点图（子节点） Module Graph

In [ ]:

# 通过net.modules()方法可以查看模型的所有模块
for module in net.modules():
    print(module)
# 通过net.named_modules()方法可以查看模型的所有模块及对应的名字
for name, module in net.named_modules():
    print(name, module)
# 通过net.children()方法可以查看模型的所有子模块
for child in net.children():
    print(child)
# 通过net.named_children()方法可以查看模型的所有子模块及对应的名字
for name, child in net.named_children():
    print(name, child)

#### 1.5 模式切换以及模型保存和加载

In [ ]:

# 通过net.train()方法可以将模型设置为训练模式
net.train()
# 通过net.eval()方法可以将模型设置为评估模式
net.eval()
# 通过net.to()方法可以将模型转移到指定的设备
net.to('cuda:0')
# 通过net.apply()方法可以对模型的所有模块进行指定的操作
def init_weights(m):
    print(m)
net.apply(init_weights)
# 通过net.zero_grad()方法可以将模型的参数的梯度清零
net.zero_grad()


In [ ]:

# 通过net.state_dict()方法可以查看模型的状态字典
print(net.state_dict())
# 通过net.load_state_dict()方法可以加载模型的状态字典
net.load_state_dict(torch.load('model.pth'))
# 通过net.save()方法可以保存模型的状态字典
torch.save(net.state_dict(), 'model.pth')
# 通过net.load()方法可以加载模型的状态字典
net.load_state_dict(torch.load('model.pth'))
